In [1]:
import pymysql
from sqlalchemy import create_engine
import pandas as pd
import numpy as np

In [2]:
import getpass  # To get the password without showing the input
password = getpass.getpass()

········


In [3]:
connection_string = 'mysql+pymysql://root:' + password + '@localhost/sakila'
engine = create_engine(connection_string)

In [4]:
# 1. Create a query or queries to extract the information you think may be relevant for building the prediction model
query='''
SELECT f.film_id, f.title, f.rental_rate, f.rental_duration, f.rating, c.name AS category, sub2.n_rentals 
FROM sakila.film f
LEFT JOIN sakila.film_category fc 
USING(film_id)
LEFT JOIN sakila.category c 
USING(category_id)
LEFT JOIN(
SELECT film_id, SUM(sub.n_rentals_by_inventory_id) AS n_rentals
FROM sakila.inventory
JOIN(
	SELECT inventory_id, COUNT(inventory_id) AS n_rentals_by_inventory_id
	FROM sakila.rental
    WHERE (YEAR(rental_date) = 2005)
	GROUP BY inventory_id) sub
USING (inventory_id)
GROUP BY (film_id)) sub2
USING (film_id);
'''

In [5]:
pd.read_sql_query(query, engine)

,film_id,title,rental_rate,rental_duration,rating,category,n_rentals
0,1,ACADEMY DINOSAUR,0.99,6,PG,Documentary,23.0
1,2,ACE GOLDFINGER,4.99,3,G,Horror,6.0
2,3,ADAPTATION HOLES,2.99,7,NC-17,Documentary,12.0
3,4,AFFAIR PREJUDICE,2.99,5,G,Horror,22.0
4,5,AFRICAN EGG,2.99,6,G,Family,11.0
...,...,...,...,...,...,...,...
995,996,YOUNG LANGUAGE,0.99,6,G,Documentary,7.0
996,997,YOUTH KICK,0.99,4,NC-17,Music,6.0
997,998,ZHIVAGO CORE,0.99,6,NC-17,Horror,8.0
998,999,ZOOLANDER FICTION,2.99,5,R,Children,17.0


In [6]:
# 2. Create a query to get the list of films and a boolean indicating if it was rented last month (May 2005). 
# This would be our target variable.
query='''
SELECT f.film_id, f.title, f.rental_rate, f.rental_duration, f.rating, c.name AS category, sub2.n_rentals, 
CASE
WHEN sub2.n_rentals>0 then "True" 
ELSE "False" 
END AS "rented_in_may" 
FROM sakila.film f
LEFT JOIN sakila.film_category fc 
USING(film_id)
LEFT JOIN sakila.category c 
USING(category_id)
LEFT JOIN(
SELECT film_id, SUM(sub.n_rentals_by_inventory_id) AS n_rentals
FROM sakila.inventory
JOIN(
	SELECT inventory_id, COUNT(inventory_id) AS n_rentals_by_inventory_id
	FROM sakila.rental
    WHERE (YEAR(rental_date) = 2005 AND month(rental_date)=5)
	GROUP BY inventory_id) sub
USING (inventory_id)
GROUP BY (film_id)) sub2
USING (film_id);
'''

In [7]:
pd.read_sql_query(query, engine)

,film_id,title,rental_rate,rental_duration,rating,category,n_rentals,rented_in_may
0,1,ACADEMY DINOSAUR,0.99,6,PG,Documentary,2.0,True
1,2,ACE GOLDFINGER,4.99,3,G,Horror,NaN,False
2,3,ADAPTATION HOLES,2.99,7,NC-17,Documentary,1.0,True
3,4,AFFAIR PREJUDICE,2.99,5,G,Horror,2.0,True
4,5,AFRICAN EGG,2.99,6,G,Family,1.0,True
...,...,...,...,...,...,...,...,...
995,996,YOUNG LANGUAGE,0.99,6,G,Documentary,NaN,False
996,997,YOUTH KICK,0.99,4,NC-17,Music,NaN,False
997,998,ZHIVAGO CORE,0.99,6,NC-17,Horror,1.0,True
998,999,ZOOLANDER FICTION,2.99,5,R,Children,1.0,True


In [8]:
# 3. Read the data into a Pandas dataframe
data=pd.read_sql_query(query, engine)
data.head()

,film_id,title,rental_rate,rental_duration,rating,category,n_rentals,rented_in_may
0,1,ACADEMY DINOSAUR,0.99,6,PG,Documentary,2.0,True
1,2,ACE GOLDFINGER,4.99,3,G,Horror,NaN,False
2,3,ADAPTATION HOLES,2.99,7,NC-17,Documentary,1.0,True
3,4,AFFAIR PREJUDICE,2.99,5,G,Horror,2.0,True
4,5,AFRICAN EGG,2.99,6,G,Family,1.0,True


In [9]:
data.shape

(1000, 8)

In [10]:
data.dtypes

film_id              int64
title               object
rental_rate        float64
rental_duration      int64
rating              object
category            object
n_rentals          float64
rented_in_may       object
dtype: object

In [12]:
# % of rented films in may
(data['rented_in_may']=='True').sum()/len(data['rented_in_may'])

0.686

In [15]:
# 4. Analyze extracted features and transform them
data = data.drop(['film_id', 'title', 'n_rentals'], axis=1)
# N_rental (number of times one specific film was rented on May, matches 100% with objective, so I had to drop it)
# I was trying to take into account that the films that were rented in May are more likely to be rented next month,
# and the number of times rented was also important.

KeyError: "['film_id' 'title' 'n_rentals'] not found in axis"

In [16]:
# this columns have numerical data but discrete with very few differente values, so I will treat them as categorical
data['rental_rate'] = data['rental_rate'].astype('object')
data['rental_duration'] = data['rental_duration'].astype('object')

In [17]:
data.isna().sum()

rental_rate        0
rental_duration    0
rating             0
category           0
rented_in_may      0
dtype: int64

In [21]:
data['rented_in_may'].value_counts()

True     686
False    314
Name: rented_in_may, dtype: int64

In [22]:
data.head()

,rental_rate,rental_duration,rating,category,rented_in_may
0,0.99,6,PG,Documentary,True
1,4.99,3,G,Horror,False
2,2.99,7,NC-17,Documentary,True
3,2.99,5,G,Horror,True
4,2.99,6,G,Family,True


In [23]:
# Build X and y
y = data['rented_in_may']
X = data.drop(['rented_in_may'], axis=1)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1337)

In [24]:
X_train_num = X_train.select_dtypes(include = np.number)
X_train_cat = X_train.select_dtypes(include = np.object)

C:\Users\Usuario\AppData\Local\Temp/ipykernel_3868/182726265.py:2: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  X_train_cat = X_train.select_dtypes(include = np.object)


In [25]:
X_train_categorical = pd.get_dummies(X_train_cat, 
                             columns=['rental_rate', 'rental_duration', 'rating', 'category'],
                             drop_first=True)
X_train_categorical

,rental_rate_2.99,rental_rate_4.99,rental_duration_4,rental_duration_5,rental_duration_6,rental_duration_7,rating_NC-17,rating_PG,rating_PG-13,rating_R,...,category_Drama,category_Family,category_Foreign,category_Games,category_Horror,category_Music,category_New,category_Sci-Fi,category_Sports,category_Travel
46,0,1,0,1,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,0,0
789,0,0,0,0,1,0,0,0,1,0,...,1,0,0,0,0,0,0,0,0,0
722,1,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
283,0,1,0,1,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
39,0,0,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167,0,0,0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
232,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
860,0,1,0,0,0,0,0,0,0,1,...,0,0,0,1,0,0,0,0,0,0
189,0,1,0,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [26]:
X_train_transformed = np.concatenate([X_train_num, X_train_categorical], axis=1)
X_train_transformed.shape

(800, 25)

In [27]:
# 5. Create a logistic regression model to predict this variable from the cleaned data.
from sklearn.linear_model import LogisticRegression
classification = LogisticRegression(random_state=0, solver='lbfgs',
                  multi_class='multinomial').fit(X_train_transformed, y_train)

In [28]:
X_test_num = X_test.select_dtypes(include = np.number)

In [29]:
X_test_cat = X_test.select_dtypes(include = np.object)
X_test_categorical = pd.get_dummies(X_test_cat, 
                            columns=['rental_rate', 'rental_duration', 'rating', 'category'],
                            drop_first=True)
# display(list(zip(list(X_train_categorical.columns),list(X_test_categorical.columns))))

C:\Users\Usuario\AppData\Local\Temp/ipykernel_3868/2621457530.py:1: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  X_test_cat = X_test.select_dtypes(include = np.object)


In [30]:
X_test.head()
X_test_transformed = np.concatenate([X_test_num, X_test_categorical], axis=1)

In [31]:
predictions = classification.predict(X_test_transformed)
classification.score(X_test_transformed, y_test)

0.695

In [32]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, predictions)

array([[  2,  60],
       [  1, 137]], dtype=int64)

In [33]:
# The sistem seems to have a high accuracy to predict rented films, but fails completely when trying to predict non rented films

In [34]:
# 6. Evaluate the results. I will try to predict June rentals
query='''
SELECT f.film_id, f.title, f.rental_rate, f.rental_duration, f.rating, c.name AS category, sub2.n_rentals, 
CASE
WHEN sub2.n_rentals>0 then "True" 
ELSE "False" 
END AS "rented_in_june" 
FROM sakila.film f
LEFT JOIN sakila.film_category fc 
USING(film_id)
LEFT JOIN sakila.category c 
USING(category_id)
LEFT JOIN(
SELECT film_id, SUM(sub.n_rentals_by_inventory_id) AS n_rentals
FROM sakila.inventory
JOIN(
	SELECT inventory_id, COUNT(inventory_id) AS n_rentals_by_inventory_id
	FROM sakila.rental
    WHERE (YEAR(rental_date) = 2005 AND month(rental_date)=6)
	GROUP BY inventory_id) sub
USING (inventory_id)
GROUP BY (film_id)) sub2
USING (film_id);
'''

pd.read_sql_query(query, engine)

data_june=pd.read_sql_query(query, engine)
data_june.head()

,film_id,title,rental_rate,rental_duration,rating,category,n_rentals,rented_in_june
0,1,ACADEMY DINOSAUR,0.99,6,PG,Documentary,3.0,True
1,2,ACE GOLDFINGER,4.99,3,G,Horror,NaN,False
2,3,ADAPTATION HOLES,2.99,7,NC-17,Documentary,1.0,True
3,4,AFFAIR PREJUDICE,2.99,5,G,Horror,2.0,True
4,5,AFRICAN EGG,2.99,6,G,Family,2.0,True


In [35]:
data_june = data_june.drop(['film_id', 'title', 'n_rentals'], axis=1)
data_june['rental_rate'] = data_june['rental_rate'].astype('object')
data_june['rental_duration'] = data_june['rental_duration'].astype('object')

In [37]:
# % of rented films in june
(data_june['rented_in_june']=='True').sum()/len(data_june['rented_in_june'])

0.9

In [38]:
# Build X and y
y_june = data_june['rented_in_june']
X_june = data_june.drop(['rented_in_june'], axis=1)

X_test=X_june
y_test=y_june

X_test_num = X_test.select_dtypes(include = np.number)

X_test_cat = X_test.select_dtypes(include = np.object)
X_test_categorical = pd.get_dummies(X_test_cat, 
                            columns=['rental_rate', 'rental_duration', 'rating', 'category'],
                            drop_first=True)

X_test_transformed = np.concatenate([X_test_num, X_test_categorical], axis=1)

predictions = classification.predict(X_test_transformed)
classification.score(X_test_transformed, y_test)

C:\Users\Usuario\AppData\Local\Temp/ipykernel_3868/1826535208.py:10: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  X_test_cat = X_test.select_dtypes(include = np.object)


0.884

In [39]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, predictions)

array([[  6,  94],
       [ 22, 878]], dtype=int64)

In [40]:
# The score is 88.4%, but again fails to predict non rented films. May be this time the score is higher 
# than may due to the high rate of films rented (90%)